Лабораторная работа №3
Выполнил студент группы БСТ2001 Кувшинов М.И.

Задание №1

1 Реализовать методы поиска подстроки в строке.
2 Добавить возможность ввода строки и подстроки с клавиатуры.
3 Предусмотреть возможность существования пробела.
4 Реализовать возможность выбора опции чувствительности или нечувствительности к регистру.
5 Оценить время работы каждого алгоритма поиска и сравнить его со временем работы стандартной функции поиска, используемой в выбранном языке программирования.

In [19]:
#Алгоритм Кнута-Морриса-Пратта
def kmp(subStr):
    prefFunc = [0]*len(subStr)
    for i in range(1,len(subStr)):
        k = prefFunc[i-1]
        while k > 0 and subStr[k] != subStr[i]:
            k = prefFunc[k-1]
        if subStr[k] == subStr[i]:
            k = k + 1
        prefFunc[i] = k
    return prefFunc

def kmp_search(subStr,line,case,space):
    space = space.lower()
    case = case.lower()
    _subStr = subStr
    _line = line
    if case == "no":
        subStr = _subStr.lower()
        line = _line.lower()
    if space =="no":
        _subStr = _subStr.replace(" ","")
        _line = _line.replace(" ","")
    index = -1
    f = kmp(subStr)
    k = 0
    for i in range(len(line)):
        while k > 0 and subStr[k] != line[i]:
            k = f[k-1]
        if subStr[k] == line[i]:
            k = k + 1
        if k == len(subStr):
            index = i - len(subStr) + 1
            break
    if space =="no":
            m=0
            n=0
            spacesNumber = 0
            while m<=k+1 and n<=len(line):
                if _line[m]==line[n]:
                    m+=1
                    n+=1
                else:
                    if line[n]==" ":
                        spacesNumber +=1
                        n+=1
            return index+spacesNumber
    else: 
        return index
#Упрощенный алгоритм Бойера-Мура
def displacement(subStr):
    table = [len(subStr)]*256
    for i in range(len(subStr) - 1):
        table[ord(subStr[i])] = len(subStr) - 1 - i
    return table

def bm_search(subStr, line,case,space):
    space = space.lower()
    case = case.lower()
    _subStr = subStr
    _line = line
    if case == "no":
        _subStr = _subStr.lower()
        _line = _line.lower()
    if space =="no":
        _subStr = _subStr.replace(" ","")
        _line = _line.replace(" ","")
        
    table = displacement(_subStr)
    i = len(_subStr) - 1
    j = i
    k = i
    while j >= 0 and i <= len(_line) - 1:
        j = len(_subStr) - 1
        k = i
        while j >= 0 and _line[k] == _subStr[j]:
            k-=1
            j-=1
        i += table[ord(_line[i])]
    
    if k >= len(_line) - len(_subStr):
        return -1
    else :
        if space =="no":
            m=0
            n=0
            spacesNumber = 0
            while m<=k+1 and n<=len(line):
                if _line[m]==line[n]:
                    m+=1
                    n+=1
                else:
                    if line[n]==" ":
                        spacesNumber +=1
                        n+=1
            return k+1+spacesNumber
        else: 
            return k+1



In [21]:
import timeit
import copy
string = input("input the string: ")
string1 = copy.copy(string)
subStr = input("input the substring: ")
case = input("Case sensitivity, write Yes or No: ")
space = input("Space sensitivity, write Yes or No: ")
startTime = timeit.default_timer()
print(kmp_search(subStr,string,case,space))
print("Алгоритм Кнута-Морриса-Пратта длится " + str(timeit.default_timer() - startTime) + " сек")
startTime = timeit.default_timer()
print(bm_search(subStr,string1,case,space))
print("Упрощенный алгоритм Бойера-Мура длится " + str(timeit.default_timer() - startTime) + " сек")
startTime = timeit.default_timer()
print(string.find(subStr))
print("Встроенный поиск длится " + str(timeit.default_timer() - startTime) + " сек")

input the string: aqerdv qdrv qwdrv WDV wdv
input the substring: WDV
Case sensitivity, write Yes or No: yes
Space sensitivity, write Yes or No: yes
18
Алгоритм Кнута-Морриса-Пратта длится 0.00029640000002473244 сек
18
Упрощенный алгоритм Бойера-Мура длится 0.00029180000001360895 сек
18
Встроенный поиск длится 0.00021120000019436702 сек


In [3]:
class Position:
    def __init__(self, position, start_distance,finish_distance):
        self.position = position
        self.start_distance = start_distance
        self.finish_distance = finish_distance

    def __str__(self):
        return '\n'.join((N*'{:3}').format(*[i%(N*N) for i in self.position[i:]]) for i in range(0, N*N, N))

    # Переопределяем метод less then для работы PriorityQueue
    def __lt__(self, other):
        return self.start_distance+self.finish_distance < other.start_distance+other.finish_distance

In [4]:
from queue import PriorityQueue

N = 4
def shifts(position):
    zeroPosition = position.index(0)
    i, j = divmod(zeroPosition, N)    
    displacement = []
    if i > 0: displacement.append(-N)     # вверх
    if i < N - 1: displacement.append(N)  # вниз
    if j > 0: displacement.append(-1)     # влево
    if j < N - 1: displacement.append(1)  # вправо
    for offset in displacement:
        swap = zeroPosition + offset
        yield tuple(position[swap] if x==zeroPosition else position[zeroPosition] if x==swap else position[x] for x in range(N*N))

def parityOfPairs(state):
    countOfPairs = 0
    for i in range(len(state)-1):
        if state[i] > state[i+1]:
            countOfPairs +=1
    return countOfPairs % 2

In [5]:
def fifteen(startState):
    terminalState = (1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0)
    if parityOfPairs(startState)==0:
        print("Нет решений")
    else:
        startState= tuple(startState)
        p = Position(startState, 0,0)
        print(p)
        print()
        fieldStates= PriorityQueue()
        fieldStates.put(p)
        closePoints = set([p])
        parents = {p.position: None}
   
        while p.position != terminalState:
            p =fieldStates.get()

            for k in shifts(p.position):
                count= 0
                if k not in closePoints:
                    for m in range(len(k)):
                        if k[m] != terminalState[m]:
                            count+=1

                    fieldStates.put(Position(k, p.start_distance +1,p.finish_distance+count))
                    parents[k] = p
                    closePoints.add(k)

        path = []
        x = p
        previous = p
        while p.position != startState:
            p = parents[p.position]
            number = p.position[previous.position.index(0)]
            path.append(number)
            previous = p
        path.reverse()

        print(path)
        print(x)

In [6]:
startState = [1, 2, 3, 4, 5, 6, 7, 8, 13, 9,11,12,10,14,15,0]
fifteen(startState)

  1  2  3  4
  5  6  7  8
 13  9 11 12
 10 14 15  0

[15, 14, 10, 13, 9, 10, 14, 15]
  1  2  3  4
  5  6  7  8
  9 10 11 12
 13 14 15  0
